## Calculation on real operating pressure across reverse osmosis membrane


Hello! You can fully run this notebook on Google Colab for free! The lines below install the packages used. Once installed, you could skip them on the same session

In [1]:
!pip install condacolab #I use Google Colab because it eases installation
import condacolab
condacolab.install_miniforge()
!conda install -c conda-forge reaktoro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached condacolab-0.1.3-py3-none-any.whl (6.8 kB)
✨🍰✨ Everything looks OK!
Solving environment: - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



#Introduction

In [2]:
from IPython.core.display import display, HTML, Image  

Image(url= "https://cdn11.bigcommerce.com/s-g6xc349ydi/images/stencil/original/products/113/4759/sistema-osmosis-inversa-industrial-para-agua-salobre-imagen1__38869.1603754875.jpg?c=2", width = 600)


A typical industrial-scale reverse osmosis device is shown above. The five tubes are rolled, intending to increase the radial area along the axial axis. Inside, there is an engineered material is wound along the center to give as many layers of membrane as possible.  This results in a greater amount of permiate per square meter installed. 

This example will deal with a small device meant for households.

##Data entry

In [3]:
import reaktoro as rkt
from reaktoro import *
import pandas as pd

During this part of the script, a black box mass balance will be performed, dividing an input stream into two outputs. Reverse osmosis membranes selectively permeate the water, retaining the concentrated salt. By choosing the two permiation fractions, one for H2O and another for all salts, the system is fully defined.

A typical tap composition was taken from database: 
https://www.open.edu/openlearn/mod/oucontent/view.php?printable=1&id=20880

The values of the permiating fractions will be taken from the manual:
https://www.purewaterproducts.com/img/docs/manuals/FLEXEON-210-LT-SERIES-USERS-MANUAL.pdf

The goal of the notebook is to match the calulatated operating pressure with such that the manufacturer indicates in that reference.

In [4]:
Water = pd.DataFrame(index = ['Ca+2','Mg+2','Na+','Cl-','NO3-','SO4-2'], columns=["loading", "wt_fraction_dry"])

Water.loc["Ca+2" ,  "loading"]   =  102  #mg/L
Water.loc["Mg+2",   "loading"]   =  8.81 #mg/L
Water.loc["Na+" ,   "loading"]   =  49.1 #mg/L
Water.loc["Cl-",    "loading"]   =  73.9 #mg/L
Water.loc["NO3-",   "loading"]   =  20.6 #mg/L
Water.loc["SO4-2",  "loading"]   =  120  #mg/L

Water.wt_fraction_dry = (Water['loading'] / Water['loading'].sum())

#print(Water)

In [5]:
T_env = 25 #C
T_env = T_env + 273.15 #K

P_env = 3 #atm #The water is entering pressurized from the water network

In [6]:
mass_flow_solution = 1/60 #kg/s
TDS = 550 #mg_solids_kg_solution

liquid_permiating_fraction = 0.47 #Mass fraction of water that permiates throught the membrane
solute_permiating_fraction = 0.01  #Mass fraction of solids that permiates though the membrane 

print("An inflow of brackish water enters a Reverse Osmosis Unit", T_env - 273.15 , "C", "and", P_env, "atm")
print("With a Total Dissolved Solids (TDS) of", TDS, "mg/kg")

An inflow of brackish water enters a Reverse Osmosis Unit 25.0 C and 3 atm
With a Total Dissolved Solids (TDS) of 550 mg/kg


#Problem definition on Reaktoro

To solve the system defined above, it is necessary to define the problem in terms that Reaktoro can handle.

Reaktoro will then minimize the Gibbs free energy of the system for a set composition, flow, pressure and temperature.

Initially, all streams will be at ambient temperature and pressure, from which a Gibbs free energy balance will be performed to account for the demixing power requirements.


In [7]:
#Initialize a thermodynamic database

db = SupcrtDatabase("supcrtbl")

#Define the chemical system

species = ("H2O(aq) H+ OH- Na+ Cl- Ca+2 Mg+2 NO3- SO4-2")
species_list = species.split()

solution = AqueousPhase(species)

# solution.setActivityModel(ActivityModelHKF())
solution.setActivityModel(ActivityModelPitzerHMW())

system = ChemicalSystem(db, solution)

In [8]:
inlet_state = ChemicalState(system)
inlet_state.temperature(T_env, "K")
inlet_state.pressure(P_env, "atm")
inlet_state.set("H2O(aq)", mass_flow_solution * (1e6 - TDS)/1e6 , "kg")

for ion, row in Water.iterrows():
  inlet_state.set(ion, Water.loc[ion, "wt_fraction_dry"] * mass_flow_solution * TDS/1e6,   "kg")

permiate_state = ChemicalState(system)
permiate_state.temperature(T_env, "K")
permiate_state.pressure(P_env, "atm")
permiate_state.set("H2O(aq)", mass_flow_solution * liquid_permiating_fraction * (1e6 - TDS)/1e6 , "kg")

for ion, row in Water.iterrows():
  permiate_state.set(ion, Water.loc[ion, "wt_fraction_dry"] * mass_flow_solution * solute_permiating_fraction * TDS/1e6,   "kg")

retentate_state = ChemicalState(system)
retentate_state.temperature(T_env, "K")
retentate_state.pressure(P_env, "atm")
retentate_state.set("H2O(aq)", mass_flow_solution * (1- liquid_permiating_fraction) * (1e6 - TDS)/1e6 , "kg")

for ion, row in Water.iterrows():
  retentate_state.set(ion, Water.loc[ion, "wt_fraction_dry"] * mass_flow_solution * (1 - solute_permiating_fraction ) * TDS/1e6,   "kg")

In [9]:
solver = EquilibriumSolver(system)

result = solver.solve(inlet_state)  # equilibrate the `state` object!
assert result.optima.succeeded

result = solver.solve(permiate_state)  # equilibrate the `state` object!
assert result.optima.succeeded

result = solver.solve(retentate_state)  # equilibrate the `state` object!
assert result.optima.succeeded

In [10]:
def gibbsEnergy(props):
    u = props.speciesChemicalPotentials()
    n = props.speciesAmounts()
    G = 0.0
    for i in range(len(n)):
        G += n[i] * u[i]
    return G
    
Gibbs_Free_Energy_inlet = gibbsEnergy(inlet_state.props())
Gibbs_Free_Energy_permiate = gibbsEnergy(permiate_state.props())
Gibbs_Free_Energy_retentate = gibbsEnergy(retentate_state.props())

In [11]:
DG = ( Gibbs_Free_Energy_retentate + Gibbs_Free_Energy_permiate - Gibbs_Free_Energy_inlet )

print("The demixing Gibbs Free Energy is +", str(DG), "W", ", which will have to be provided by means of pumping the inlet")

The demixing Gibbs Free Energy is + 0.266616 W , which will have to be provided by means of pumping the inlet


#Osmotic Pressure calculation

Once the demixing power has been calculated, the inlet must be pumped so that it retains the necessary Gibbs free energy that satisfies the total DG balance. This is DG_total = DG_pumping + DG_RO = 0; DG_pumping = -DG_RO.

The pressure is iterated until the DG constraint is reached. Under these conditions, the elevated pressure is the so-called osmotic pressure. This is the minimum pressure to be exerted at the inlet to perform the demixing step.

In [12]:
props = ChemicalProps(inlet_state)

specs = EquilibriumSpecs(system)
specs.temperature()

idxDG = specs.addInput("DG")

GibbsConstraint = ConstraintEquation()
GibbsConstraint.id = "DGConstraint" 
GibbsConstraint.fn = lambda props, w: float(Gibbs_Free_Energy_retentate) + float(Gibbs_Free_Energy_permiate) - gibbsEnergy(props) - w[idxDG]
specs.addConstraint(GibbsConstraint)

solver = EquilibriumSolver(specs)

options = EquilibriumOptions()
options.optima.output.active = True
solver.setOptions(options)

conditions = EquilibriumConditions(specs)

conditions.temperature(T_env, "K")
conditions.set("DG", 0)  

conditions.setLowerBoundPressure(1, "bar")
conditions.setUpperBoundPressure(10, "bar")  

result = solver.solve(inlet_state, conditions)
assert result.optima.succeeded

atm_to_Pa = 101325 #Pa/atm

print(f"The required osmotic pressure to be exerted for demixing is {inlet_state.props().pressure()/atm_to_Pa} atm")
print("The Gibbs Free Energy Balance openness is", Gibbs_Free_Energy_retentate + Gibbs_Free_Energy_permiate - gibbsEnergy(inlet_state.props()), "W")



The required osmotic pressure to be exerted for demixing is 3.15751 atm
The Gibbs Free Energy Balance openness is -5.82077e-11 W


#Accounting for real operation (non-idealities)

As any real machine, Reverse Osmosis does not have input-output 100% efficiency when converting pressure energy and selectively permiating. A great chunk of the mechanical power is degraded into thermal power via friction. The following lines correspond to the calculation of the adiabatic shooting that the two outlets undergo, raising their temperature by means of irreversible heat generation. 

In [13]:
k = 1e-6 #Convergence factor

P_osm = inlet_state.props().pressure()/atm_to_Pa

Operating_pressure = 2*P_osm

Efficiency_low_P_RO =  0.05 #https://www.mdpi.com/1996-1073/14/8/2275 


for k in range(1000):
         
  Volumetric_flow = inlet_state.props().mass()/ inlet_state.props().density() #m3/s

  Osmotic_power = (P_osm - P_env) *  atm_to_Pa * Volumetric_flow   #W

  Real_power = (Operating_pressure - P_env) *  atm_to_Pa * Volumetric_flow  #W

  Irreversible_heat_generation = (Real_power - Osmotic_power) #W

  Efficiency_calculated = Osmotic_power / Real_power

  Error_eff = (Efficiency_calculated - Efficiency_low_P_RO)*(Efficiency_calculated - Efficiency_low_P_RO)

  Operating_pressure = Operating_pressure - k * Error_eff


#print(Operating_pressure, Error_eff)

print("By comparing a typical efficiency with the one calculated, the real operating is", Operating_pressure, "atm")

print("Efficiency taken from: https://www.mdpi.com/1996-1073/14/8/2275")

print("A commercial RO membrane meant for household would require a pressure in the range of 3 - 8  atm")

print("Household RO device specs taken from: Page 17 - https://www.purewaterproducts.com/img/docs/manuals/FLEXEON-210-LT-SERIES-USERS-MANUAL.pdf")

print("The real power is", Real_power, "W_PVwork")

print("The irreversible heat generation is", (Real_power - Osmotic_power), "W_thermal")

By comparing a typical efficiency with the one calculated, the real operating is 6.15791 atm
Efficiency taken from: https://www.mdpi.com/1996-1073/14/8/2275
A commercial RO membrane meant for household would require a pressure in the range of 3 - 8  atm
Household RO device specs taken from: Page 17 - https://www.purewaterproducts.com/img/docs/manuals/FLEXEON-210-LT-SERIES-USERS-MANUAL.pdf
The real power is 5.3453 W_PVwork
The irreversible heat generation is 5.07868 W_thermal


Now, as we know that there is a degradated PV power going to thermal energy, the outlet streams temperatures will be raised accordingly. This is done with a fixed point convergence method, iterating the adiabatic temperature to satisfy an enthalpy balance.


In [14]:
T_iter = 25 #C
T_iter = T_iter + 273.15 #K

k = 1e-3

for iter in range(2000):

  permiate_state.temperature(T_iter, "K")
  retentate_state.temperature(T_iter, "K")

  mcp_permiate = permiate_state.props().heatCapacityConstP()   #W/K
  mcp_retentate = retentate_state.props().heatCapacityConstP()  #W/K

  error_T =  (T_iter - T_env) * ( mcp_permiate + mcp_retentate ) - Irreversible_heat_generation #W

  T_iter = T_iter - k * error_T 

print("Temperature overshooting raises up permiate and retantate by",  T_iter - T_env, "C")

print("Leading an energy balance error of", error_T, "kW")

print("Permiate and retantate exit at", T_iter - 273.15, "C", "instead of", T_env - 273.15, "C")

Temperature overshooting raises up permiate and retantate by 0.072943 C
Leading an energy balance error of -2.79634e-11 kW
Permiate and retantate exit at 25.0729 C instead of 25.0 C


##Concluding remarks

For the set of salt and liquid permeating pass, inlet temperature, inlet pressure and energy efficiency, the operating pressure is within the range provided by the manufacturer.

Adibatic overshooting takes most of the mecanical power, once degradated via friction. However, as the heat capacities of the aqueous outlets are high, temperature raising is within the 1e-2 C order of magnitude
